In [21]:
import numpy as np
import h5py
import torch
#from torch.utils.data import Dataset, DataLoader
# import torch.utils.data as data
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

# Helper Function

In [22]:
class EEGDataset(data_utils.Dataset):
    """EEG dataset."""
    def __init__(self, X, y):
        self.X = X;
        self.y = y;
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
class Flatten(nn.Module):
    def forward(self, x):
        N, E, T = x.size() 
        return x.view(N, -1)

# Preprocess the Data

In [23]:
A01T = h5py.File('A01T_slice.mat', 'r')
print (A01T)
X = np.copy(A01T['image'])
y = np.copy(A01T['type'])
y = y[0,0:X.shape[0]:1]
y = np.asarray(y, dtype=np.int32)
y = y - 769;

<HDF5 file "A01T_slice.mat" (mode r)>


In [24]:
data = EEGDataset(X,y)
dset = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [len(data)-100, 50, 50])
dataloaders = {x: data_utils.DataLoader(dset[x], batch_size=10, shuffle=True, num_workers=1) for x in ['train', 'val', 'test']}

# Testing the Dataloader

In [26]:
# Torch.DoubleTensor
for i_batch, sample_batched in enumerate(dataloaders['train']):
    X, y = sample_batched
    print (X)


( 0  ,.,.) = 
   19.2871   17.2852   15.1855  ...   -35.1562  -35.5957  -41.6992
   18.0176   13.2324   14.5508  ...   -21.7773  -21.8262  -27.5879
   18.5059   17.9199   15.2344  ...   -22.4121  -22.9980  -27.0508
              ...                ⋱                ...             
   -0.9766    3.4180    0.4883  ...    13.1836   10.2539    4.3945
   19.5312   15.1367   20.5078  ...  -112.7930 -117.6758 -127.4414
    7.8125   -4.3945    4.3945  ...    -3.4180   -2.9297   -6.8359

( 1  ,.,.) = 
    3.8086    4.8340   -0.4883  ...    15.0391   10.2539   -1.3672
    2.2461    5.2734    1.1719  ...    13.6230    3.3691   -3.7109
    4.6875    6.1035    1.1719  ...    14.1113    9.4238   -0.7324
              ...                ⋱                ...             
    8.3008   11.7188  -10.2539  ...     9.2773   -5.8594   -1.4648
    4.3945    8.3008   -0.9766  ...    10.2539    4.3945    0.9766
    3.9062    3.4180   -3.4180  ...    -0.4883   -0.4883    0.9766

( 2  ,.,.) = 
   -7.6660   -2.2


( 0  ,.,.) = 
    7.3730    6.9336    4.2480  ...    -0.3418    2.3926    5.4688
   -1.5137    3.4668    1.5625  ...     1.9531    4.1992    3.9551
    3.3691    2.7344    0.6836  ...    -3.6621   -0.8789    2.5391
              ...                ⋱                ...             
    4.8828    2.9297    4.8828  ...     0.9766    5.8594    5.8594
    7.3242    9.2773   12.6953  ...     6.3477    8.7891    3.4180
    7.8125    7.3242    8.7891  ...     3.9062    6.8359   10.2539

( 1  ,.,.) = 
   -7.7148   -1.5137   -8.2031  ...     5.2246    5.7129    7.8125
  -10.8398   -4.2480   -5.7129  ...    -1.1230   -4.8340   -0.1465
   -8.6426   -3.2227   -8.5938  ...     4.4434    2.8809    5.7617
              ...                ⋱                ...             
  -11.7188   -0.9766   -1.9531  ...     6.3477    5.3711   11.2305
  -13.1836   -0.4883   -2.9297  ...     9.7656   11.2305   16.6016
    1.9531    2.9297   -0.9766  ...     0.4883   -0.9766    3.4180

( 2  ,.,.) = 
    4.5410   10.6

# Define the Model

In [18]:
dtype = torch.DoubleTensor
#N, E, T= 100, 25, 500
#C = 4
N, E, T= 288, 25, 1000
C = 4;
model = nn.Sequential(
    nn.LSTM(E,C,2)
)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Test Training

In [20]:
for i_batch, sample_batched in enumerate(dataloaders['train']):
        X_train, y_train = sample_batched
        print (X_train)
        X_train = Variable(X_train)
        y_train = Variable(y_train)
        #dtype = torch.FloatTensor
        #X_train.type(dtype)
        #y_train.type(dtype)
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train.type(torch.LongTensor))
        print('(%d batch) loss: %f' % (i_batch, loss))
        # Do backpropagation
        model.zero_grad()
        loss.backward()
        # Update the parameters
        optimizer.step()


( 0  ,.,.) = 
    9.1309    5.9570    6.8848  ...    -1.1719   -2.2461   -3.7109
   10.1562    3.8574    9.1797  ...     1.3672   -2.4414   -4.7363
   12.2070    8.2520   10.0586  ...    -3.9551   -5.2246   -7.1777
              ...                ⋱                ...             
    6.3477   -8.3008   -3.9062  ...     4.3945    1.9531    5.8594
   20.9961   16.1133   19.5312  ...     5.8594    1.9531    7.8125
   -3.9062   -4.3945    3.4180  ...     3.9062    2.9297    5.3711

( 1  ,.,.) = 
    6.3965   -2.4902   -0.5371  ...    13.5254   18.4570   19.1406
    3.0762   -1.5137    1.1719  ...     9.6680   14.9414   20.2148
    7.1777   -1.0254    1.3672  ...    10.8887   17.7734   20.1172
              ...                ⋱                ...             
    4.8828    1.9531    4.3945  ...     9.2773   10.7422   10.7422
    6.3477    1.9531    1.4648  ...    13.1836   14.6484   17.0898
   15.1367    0.4883    2.9297  ...     3.4180    1.9531    9.2773

( 2  ,.,.) = 
   -2.6855   -2.9

NameError: name 'fn' is not defined

# Training

In [16]:
num_epochs = 2
for epoch in range(num_epochs):  # loop over the dataset multiple times
    for i_batch, sample_batched in enumerate(dataloaders['train']):
        X_train, y_train = sample_batched
        X_train = Variable(X_train)
        y_train = Variable(y_train)
        #dtype = torch.FloatTensor
        #X_train.type(dtype)
        #y_train.type(dtype)
        y_pred = model(X_train)
        loss = loss_fn(y_pred, y_train.type(torch.LongTensor))
        print('(%d batch) loss: %f' % (i_batch, loss))
        # Do backpropagation
        model.zero_grad()
        loss.backward()
        # Update the parameters
        optimizer.step()
    ## Training Accuracy After 1 epoch
    y_pred_tot = []
    y_train_tot = []
    for i_batch, sample_batched in enumerate(dataloaders['val']):
        X_train, y_train = sample_batched
        X_train, y_train = Variable(X_train), Variable(y_train)
        scores = model(X_train)
        print (scores)
        y_pred_tot.append(np.argmax(scores, axis=0))
        y.train_tot.append(y_train)
    train_acc = np.mean(y_pred_tot == y_train_tot)
    ## Validation Accuracy After 1 epoch    
    y_pred_tot = []
    y_val_tot = []
    for i_batch, sample_batched in enumerate(dataloaders['val']):
        X_val, y_val = sample_batched
        X_val, y_val = Variable(X_val), Variable(y_train)
        scores = model(X_val)
        y_pred_tot.append(np.argmax(scores, axis=0))
        y.val_tot.append(y_val)
    val_acc = np.mean(y_pred_tot == y_val_tot)
    print('(Epoch %d / %d) train acc: %f; val_acc: %f' % (epoch, num_epochs, train_acc, val_acc))


(0 batch) loss: 0.042241
(1 batch) loss: 5.816192
(2 batch) loss: 0.000015
(3 batch) loss: 11.123883
(4 batch) loss: 3.250899
(5 batch) loss: 5.925397
(6 batch) loss: 5.162231
(7 batch) loss: 0.000006
(8 batch) loss: 27.439180
(9 batch) loss: 15.268533
(10 batch) loss: 0.000000
(11 batch) loss: 18.906391
(12 batch) loss: 0.000000
(13 batch) loss: 31.127072
(14 batch) loss: 14.552045
(15 batch) loss: 0.000000
(16 batch) loss: 4.447059
(17 batch) loss: 0.000001
(18 batch) loss: 7.657994
Variable containing:
 222.5675  -73.2677 -135.2047  -40.7436
 184.7829  153.4643  -14.8004 -192.5471
      nan       nan       nan       nan
 -33.8853  128.4333  124.1350 -194.3442
 213.3142  117.0116   91.9718 -125.0277
 -12.5737  147.8971 -177.1536 -107.6746
 154.4318  -88.4074   37.0218   35.3798
 -59.4713 -138.5239   13.0236   37.9769
-150.7744  100.7293 -343.2048   84.3093
-174.9866 -155.7896 -217.7266  225.0662
[torch.DoubleTensor of size 10x4]



SystemError: <built-in method gt of Variable object at 0x10a923408> returned a result with an error set